# Matching Bike GPS Points Utilizing the Pittsburgh Street Network

Data Dictionary: Bike_GP_Point: This feature class layer includes all Bike GPS points.

Trip_ID: A unique identifier for each set of GPS points that represent individual bike tracks. This field is found in the Bike_GP_Point attribute table.

START_AT: A field within the Bike_GP_Point feature class, this represents the timestamp when each GPS point was recorded. By arranging the GPS points in ascending order according to START_AT, we can determine the direction of each bike trip.
PittsburghStreets_ND: A TIGER-based network dataset accessible in the data folder,


Part 1: Connecting Jupyter Notebook to Your Personal ArcGIS Account and Creating a Function to Retrieve a List of All Bike Trip IDs.

In [ ]:
#part 1:
import arcgis
from arcgis.gis import GIS

# Connect to your ArcGIS Online or Portal for ArcGIS account
gis = GIS("https://www.arcgis.com", "User Name", "Password") # Replace with your arcgis user name and password 

# Check out the Network Analyst extension
arcgis.env.active_gis = gis
arcgis.env.process_spatial_analyst_extension = True
arcgis.env.process_network_analysis_extension = True

import arcpy

# Set up the environment
arcpy.env.workspace = r"C:\Users\YourName\MapMatching_Pionts.gdb"  # Replace with your workspace path

####Making a function to create a list of trip_Ids:
def get_unique_values(fc, field):
    # Use a set because it automatically handles duplicate values
    unique_values = set()
    # Use a SearchCursor to iterate through the rows in the feature class
    with arcpy.da.SearchCursor(fc, [field]) as cursor:
        for row in cursor:
            unique_values.add(row[0])

    return unique_values
####end of function

trip_ids = get_unique_values("Bike_GP_Point", "TRIP_ID")
trip_ids


Part 2: This section entails identifying of the street network and generating the corresponding route for each unique trip ID set of points. In the final polyline attribute table, you can ascertain the path/track length for each cyclist, each denoted by a unique trip ID. This script employs the Pittsburgh tiger-based network, which is free to download and use. To generate a path that is compatible with bike travel mode, the "UTurn_policy" is set to "ALLOW_UTURNS" in the arcpy.na.MakeRouteLayer function. It's important to note that the utilized Pittsburgh network lacks the precision of the ArcGIS online server network; however, using the ArcGIS online network incurs credit charges.

In [ ]:
# Part 2:
# Set up the input parameters
network_dataset = r"C:\Users\Data\PittsburghStreets\PittsburghStreets_ND" #Replace it with the file path of PittsburghStreets_ND on your computer.
Org_stops_layer = "Bike_GP_Point"
impedance_attribute = "Minutes"
UTurn_policy        = "ALLOW_UTURNS" # since we have bike GPS points

for ids in trip_ids:

    # Delete the "stops_layer" and "NSStop_Layer" if it exists
    if arcpy.Exists("stops_layer"):
        arcpy.management.Delete("stops_layer")
    if arcpy.Exists("NSstops_layer"):
        arcpy.management.Delete("NSstops_layer")

    # Create a feature layer for the stops, filtered by the current trip id
    where_clause = f"TRIP_ID = {ids}"
    arcpy.management.MakeFeatureLayer(Org_stops_layer, "NSstops_layer", where_clause)
    arcpy.management.Sort("NSstops_layer", "stops_layer", [["START_AT", "ASCENDING"]])

    route_layer_name = f"Route_{ids}"

    # Delete the layer if it exists
    if arcpy.Exists(route_layer_name):
        arcpy.management.Delete(route_layer_name)

    # Create a Route layer
    arcpy.na.MakeRouteLayer(network_dataset, route_layer_name, impedance_attribute, UTurn_policy)

    # Get the sublayers from the Route layer
    sublayer_names = arcpy.na.GetNAClassNames(route_layer_name)

    # Set up the sublayer names
    stops_sublayer = sublayer_names["Stops"]
    routes_sublayer = sublayer_names["Routes"]

    # Load the stops into the Stops sublayer
    arcpy.na.AddLocations(route_layer_name, "Stops", "stops_layer")

    # Solve the route
    arcpy.na.Solve(route_layer_name)

    # Add a new field to the route sublayer
    arcpy.AddField_management("Routes", "Trip_ID", "TEXT")
    arcpy.CalculateField_management(routes_sublayer, "Trip_ID", f"'{ids}'", "PYTHON3")

    # Save the output route to a separated folder:
    output_route_fc = rf"C:\Users\YourName\Matched_Tracks\Route_{ids}" # replace this address with a new folder path and then add Route_{id} to the end of path 
    arcpy.management.CopyFeatures("Routes", output_route_fc)

    print("Generated Route Name:", output_route_fc)

    # Delete stops_layer:
    arcpy.management.Delete("stops_layer")
    arcpy.management.Delete("NSstops_layer")

    # Delete the feature data set:
    route_dataset = arcpy.ListDatasets()[0]
    arcpy.Delete_management(route_dataset)

arcpy.env.workspace = r"C:\Users\YourName\Matched_Tracks" # replace it with a new folder path you created above
All_Routes = arcpy.ListFeatureClasses()
arcpy.management.Merge(All_Routes, "Merged_Routes")

